In [1]:
import math
import json

import keras
import librosa
import os

import numpy
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
import ast
print(tf.config.list_physical_devices('GPU'))
pd.set_option('display.max_columns', None)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
tracks = pd.read_csv(r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_metadata\tracks.csv")
genres = pd.read_csv(r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_metadata\genres.csv")
features = pd.read_csv(r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_metadata\features.csv")


C:\Users\dpetr\AppData\Local\Temp\ipykernel_3660\1455218520.py:1: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv(r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_metadata\tracks.csv")
C:\Users\dpetr\AppData\Local\Temp\ipykernel_3660\1455218520.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,

In [53]:
small_tracks = tracks.loc[tracks['set.1'] == 'small']
small_tracks.reset_index()
small_subset_track_ids = small_tracks.iloc[:,0].reset_index(drop=True)
small_subset_track_ids

0            2
1            5
2           10
3          140
4          141
         ...  
7995    154308
7996    154309
7997    154413
7998    154414
7999    155066
Name: Unnamed: 0, Length: 8000, dtype: object

In [58]:
features

,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,chroma_cens.9,chroma_cens.10,chroma_cens.11,chroma_cens.12,chroma_cens.13,chroma_cens.14,chroma_cens.15,chroma_cens.16,chroma_cens.17,chroma_cens.18,chroma_cens.19,chroma_cens.20,chroma_cens.21,chroma_cens.22,chroma_cens.23,chroma_cens.24,chroma_cens.25,chroma_cens.26,chroma_cens.27,chroma_cens.28,chroma_cens.29,chroma_cens.30,chroma_cens.31,chroma_cens.32,chroma_cens.33,chroma_cens.34,chroma_cens.35,chroma_cens.36,chroma_cens.37,chroma_cens.38,chroma_cens.39,chroma_cens.40,chroma_cens.41,chroma_cens.42,chroma_cens.43,chroma_cens.44,chroma_cens.45,chroma_cens.46,chroma_cens.47,chroma_cens.48,chroma_cens.49,chroma_cens.50,chroma_cens.51,chroma_cens.52,chroma_cens.53,chroma_cens.54,chroma_cens.55,chroma_cens.56,chroma_cens.57,chroma_cens.58,chroma_cens.59,chroma_cens.60,chroma_cens.61,chroma_cens.62,chroma_cens.63,chroma_cens.64,chroma_cens.65,chroma_cens.66,chroma_cens.67,chroma_cens.68,chroma_cens.69,chroma_cens.70,chroma_cens.71,chroma_cens.72,chroma_cens.73,chroma_cens.74,chroma_cens.75,chroma_cens.76,chroma_cens.77,chroma_cens.78,chroma_cens.79,chroma_cens.80,chroma_cens.81,chroma_cens.82,chroma_cens.83,chroma_cqt,chroma_cqt.1,chroma_cqt.2,chroma_cqt.3,chroma_cqt.4,chroma_cqt.5,chroma_cqt.6,chroma_cqt.7,chroma_cqt.8,chroma_cqt.9,chroma_cqt.10,chroma_cqt.11,chroma_cqt.12,chroma_cqt.13,chroma_cqt.14,chroma_cqt.15,chroma_cqt.16,chroma_cqt.17,chroma_cqt.18,chroma_cqt.19,chroma_cqt.20,chroma_cqt.21,chroma_cqt.22,chroma_cqt.23,chroma_cqt.24,chroma_cqt.25,chroma_cqt.26,chroma_cqt.27,chroma_cqt.28,chroma_cqt.29,chroma_cqt.30,chroma_cqt.31,chroma_cqt.32,chroma_cqt.33,chroma_cqt.34,chroma_cqt.35,chroma_cqt.36,chroma_cqt.37,chroma_cqt.38,chroma_cqt.39,chroma_cqt.40,chroma_cqt.41,chroma_cqt.42,chroma_cqt.43,chroma_cqt.44,chroma_cqt.45,chroma_cqt.46,chroma_cqt.47,chroma_cqt.48,chroma_cqt.49,chroma_cqt.50,chroma_cqt.51,chroma_cqt.52,chroma_cqt.53,chroma_cqt.54,chroma_cqt.55,chroma_cqt.56,chroma_cqt.57,chroma_cqt.58,chroma_cqt.59,chroma_cqt.60,chroma_cqt.61,chroma_cqt.62,chroma_cqt.63,chroma_cqt.64,chroma_cqt.65,chroma_cqt.66,chroma_cqt.67,chroma_cqt.68,chroma_cqt.69,chroma_cqt.70,chroma_cqt.71,chroma_cqt.72,chroma_cqt.73,chroma_cqt.74,chroma_cqt.75,chroma_cqt.76,chroma_cqt.77,chroma_cqt.78,chroma_cqt.79,chroma_cqt.80,chroma_cqt.81,chroma_cqt.82,chroma_cqt.83,chroma_stft,chroma_stft.1,chroma_stft.2,chroma_stft.3,chroma_stft.4,chroma_stft.5,chroma_stft.6,chroma_stft.7,chroma_stft.8,chroma_stft.9,chroma_stft.10,chroma_stft.11,chroma_stft.12,chroma_stft.13,chroma_stft.14,chroma_stft.15,chroma_stft.16,chroma_stft.17,chroma_stft.18,chroma_stft.19,chroma_stft.20,chroma_stft.21,chroma_stft.22,chroma_stft.23,chroma_stft.24,chroma_stft.25,chroma_stft.26,chroma_stft.27,chroma_stft.28,chroma_stft.29,chroma_stft.30,chroma_stft.31,chroma_stft.32,chroma_stft.33,chroma_stft.34,chroma_stft.35,chroma_stft.36,chroma_stft.37,chroma_stft.38,chroma_stft.39,chroma_stft.40,chroma_stft.41,chroma_stft.42,chroma_stft.43,chroma_stft.44,chroma_stft.45,chroma_stft.46,chroma_stft.47,chroma_stft.48,chroma_stft.49,chroma_stft.50,chroma_stft.51,chroma_stft.52,chroma_stft.53,chroma_stft.54,chroma_stft.55,chroma_stft.56,chroma_stft.57,chroma_stft.58,chroma_stft.59,chroma_stft.60,chroma_stft.61,chroma_stft.62,chroma_stft.63,chroma_stft.64,chroma_stft.65,chroma_stft.66,chroma_stft.67,chroma_stft.68,chroma_stft.69,chroma_stft.70,chroma_stft.71,chroma_stft.72,chroma_stft.73,chroma_stft.74,chroma_stft.75,chroma_stft.76,chroma_stft.77,chroma_stft.78,chroma_stft.79,chroma_stft.80,chroma_stft.81,chroma_stft.82,chroma_stft.83,mfcc,mfcc.1,mfcc.2,mfcc.3,mfcc.4,mfcc.5,mfcc.6,mfcc.7,mfcc.8,mfcc.9,mfcc.10,mfcc.11,mfcc.12,mfcc.13,mfcc.14,mfcc.15,mfcc.16,mfcc.17,mfcc.18,mfcc.19,mfcc.20,mfcc.21,mfcc.22,mfcc.23,mfcc.24,mfcc.25,mfcc.26,mfcc.27,mfcc.28,mfcc.29,mfcc.30,mfcc.31,mfcc.32,mfcc.33,mfcc.34,mfcc.35,mfcc.36,mfcc.37,mfcc.38,mfcc.39,mfcc.40,mfcc.41,mfcc.42

In [15]:
raw_tracks = pd.read_csv(r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_metadata\raw_tracks.csv")

In [35]:
d = ast.literal_eval(tracks['track_genres'][0])
print(d)

[{'genre_id': '21', 'genre_title': 'Hip-Hop', 'genre_url': 'http://freemusicarchive.org/genre/Hip-Hop/'}]


In [57]:
raw_tracks.head(10)
small_subset_tracks = raw_tracks.loc[raw_tracks['track_id'].isin(small_subset_track_ids)]
small_subset_tracks
# genres.head(50)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [36]:
print(d[0])

{'genre_id': '21', 'genre_title': 'Hip-Hop', 'genre_url': 'http://freemusicarchive.org/genre/Hip-Hop/'}


In [37]:
print(d[0]['genre_title'])

Hip-Hop


In [46]:
def preprocess(dataset_path, num_mfcc=40, n_fft=2048, hop_length=512, num_segment=1):
    data = {"labels":[], "mfcc":[]}
    sample_rate = 22050
    sample_per_segment = int(sample_rate*30/num_segment)

    for root, dirs, filenames in os.walk(dataset_path):
        for f in sorted(filenames):
            # file_path = str(str(root).split('\\')[-1]) + '/' + str(f)
            print("Track Name", f)

    #         try:
    #             y, sr = librosa.load(dirpath + "\\" + f, sr=sample_rate)
    #         except:
    #             print("Exception")
    #             continue
    #         for n in range(num_segment):
    #             # if (file_path == 'jazz/jazz.00055.wav'):
    #             #     print(n)
    #             mfcc = librosa.feature.mfcc(y=y[sample_per_segment*n: sample_per_segment*(n + 1)],
    #                                         sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
    #                                         hop_length=hop_length)
    #             mfcc = mfcc.T
    #             if len(mfcc) == math.ceil(sample_per_segment / hop_length):
    #                 data["mfcc"].append(mfcc.tolist())
    #                 data["labels"].append(label_idx-1)
    # return data

    preprocess(dataset_path=r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_small")

In [2]:
track_ids = np.empty(shape=(8000,), dtype=int)
counter = 0
sample_rate = 22050
num_mfcc = 40
n_fft = 2048
hop_length = 512
data = {"labels":[], "mfcc":[]}
for root, dirs, filename in os.walk(r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_small"):
    print(os.getcwd())
    for f in sorted(filename):
        # file_path = str(str(root).split('\\')[-1]) + '/' + str(f)
        if not f.endswith('mp3'):
            continue
        file_path = r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_small" + "\\" + f[0:3] + "\\" + f
        try:
            y, sr = librosa.load(file_path, sr=sample_rate, duration=29)
            # print(len(y))
            mfcc = librosa.feature.mfcc(y=y[0:(int(29 * sample_rate / hop_length) * hop_length) - 1], sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
            # print(mfcc.shape)
            mfcc = mfcc.T
            data["mfcc"].append(mfcc.tolist())
            track_id = int(f.split('.')[0])
            track_ids[counter] = track_id
            counter = counter + 1
            # print("Track Id : ", track_id)
        except:
            print("Exception")

print(counter)
counter = 0
labels = np.empty(shape=(8000,), dtype=int)
tracks = pd.read_csv(r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_metadata\raw_tracks.csv")
genres = pd.read_csv(r"C:\Users\dpetr\Desktop\sxoli\Level 3\Individual\datasets\fma_metadata\genres.csv")
for index, row in tracks.iterrows():
    if counter >= 8000:
        break
    if track_ids[counter] == row['track_id']:
        track_genre = ast.literal_eval(row['track_genres'])[0]['genre_id']
        labels[counter] = track_genre
        # print(labels[counter])
        counter = counter + 1

for index, row in genres.iterrows():
    # For each child genre search the labels and if it exists change it to its parent genre
    if row['parent'] != 0:
        for i in range(len(labels)):
            if labels[i] == row['genre_id']:
                labels[i] = row['top_level']

C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\M

C:\Users\dpetr\miniconda3\envs\tf\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
Exception
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
Exception
C:\Users\dpetr\PycharmProjects\MusicGenreClassification\MusicGenreClassification\models
C:\Users\dpe

In [ ]:
print(data['mfcc'].shape)
print(track_ids.shape)

(1248, 40)
(109727, 39)


AttributeError: 'list' object has no attribute 'shape'

In [7]:
genre, counts = np.unique(labels, return_counts=True)
print(genre)
print(counts)


[   2   10   12   15   17   21   38 1235]
[1000 1000 1000 1000 1000 1000 1000 1000]
